# settind_infra 파일의 목표
1. 수집한 데이터의 위 경도화 및 단속일시의 속성 변경 작업
- 위 경도 데이터를 활용해, 격자 별 meta 데이터를 제작하기 위함
2.  infra 데이터 프레임 종합
- 별도로 구분된 데이터 프레임의 구조를 통합함으로써 이후 meta 데이터로 변환하기 용이하게 전처리 함.

## 수집한 데이터 프레임의 위,경도화 및 단속일시의 datetime 속성 변경 작업

----

# 광진구 내 불법 주정차 단속 정보 종합

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('광진구_위경도_단속.csv')

In [5]:
df['단속구분'].unique()
# 이하 데이터는 광진구 내 단속 지점의 위,경도이며 단속 장소를 위,경도화 하여 저장한 데이터임. 따라서 위,경도화 할 수 없는 데이터는 제외되었음.

array(['스마트앱(서울시)', '안전신문고(행안부)', '생활불편앱(행안부)', 'PDA', 'CCTV', '주행형CCTV',
       '일반', '고정형CCTV', '소방서'], dtype=object)

In [6]:
df['단속일시']=pd.to_datetime(df['단속일시'])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86524 entries, 0 to 86523
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   단속일시    86524 non-null  datetime64[ns]
 1   과태료     86524 non-null  int64         
 2   단속지역    86524 non-null  object        
 3   단속장소    86524 non-null  object        
 4   위반내용    86524 non-null  object        
 5   견인지시    86524 non-null  object        
 6   단속구분    86524 non-null  object        
 7   정보      86524 non-null  object        
 8   위도      84191 non-null  float64       
 9   경도      84191 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 6.6+ MB


----

# 광진구 내 cctv 설치 정보 종합

In [8]:
a = pd.read_csv('서울시 광진구 불법주정차 위반 단속 CCTV 위치정보.csv',encoding='cp949')
b = pd.read_csv('FIXED_CCTV.csv')
# a는 광진구 내 설치된 cctv 의 설치 정보, 하지만 설치 일자는 알 수 없음
# b는 서울시 내 설치된 cctv의 설치 정보. 설치 일자도 알 수 있으므로 b에서 a 설치 지점명을 추출할 것

In [9]:
a_items = a['단속지점명'].values
target_cctv_info=b.loc[b['단속지점명'].isin(a_items)]

In [10]:
after_2022_06_11_install_cctv=  []
for i in a_items:
    if i in target_cctv_info['단속지점명'].values:
        continue
    else :
        after_2022_06_11_install_cctv.append(i)
# 포함되지 않은 cctv 정보를 별도로 저장

In [11]:
after_2022_06_11_install_cctv
# 관련 데이터의 최신 수정일자가 2022년 6월 11일이므로 이하 목록은 이후 설치된 cctv 임을 가정함

['중광초 정문 앞',
 '광장동 실내배드민턴장 앞 ',
 '신자초 정문 앞 ',
 '중광초 후문 앞',
 '자양중 정문 앞 ',
 '동의초 주변 ',
 '장순루식당 주변',
 '래미안스위트파크 103동 옆',
 '용마사거리 주변',
 '신자초등학교 후문 주변',
 '어광수의원 주변']

In [12]:
target_cctv_info = a.merge(target_cctv_info[['단속지점명','위도','경도','등록일시','단속지번주소']])

In [13]:
after_2022_06_11_install_cctv = a[a['단속지점명'].isin(after_2022_06_11_install_cctv)]

In [14]:
after_2022_06_11_install_cctv.to_csv('수정일자_이후_설치CCTV',index=False)
# 수정일자 이후 설치된 것으로 가정한 cctv의 데이터

In [15]:
target_cctv_info
# 광진구 내 58개의 cctv 주소 정보

,고정형CCTV지번주소,위도,경도,자치구,단속지점명,현장구분,등록일시,단속지번주소
0,서울 광진구 광장동 334-2,37.547790,127.106990,광진구,SK강평주유소 앞,불법주정차구역,2020-03-17 14:02:57,서울 광진구 광장동 334-2
1,서울 광진구 광장동 445-3,37.543799,127.102410,광진구,광현교회 앞,불법주정차구역,2020-03-17 14:02:57,서울 광진구 광장동 445-3
2,서울 광진구 구의동 135,37.549593,127.081780,광진구,아차산역 5번출구 앞,불법주정차구역,2020-03-17 14:02:57,서울 광진구 구의동 135
3,서울 광진구 군자동 503,37.548609,127.071072,광진구,두산위브 앞,불법주정차구역,2020-03-17 14:02:57,서울 광진구 군자동 503
4,서울 광진구 구의동 66-62,37.545348,127.088420,광진구,국민은행 앞,불법주정차구역,2020-03-17 14:02:57,서울 광진구 구의동 66-62
5,서울 광진구 구의동 68-29,37.545976,127.086327,광진구,구의사거리,불법주정차구역,2020-03-17 14:02:57,서울 광진구 구의동 68-29
6,서울 광진구 구의동 546-1,37.533558,127.093389,광진구,리젠트오피스텔건너편,불법주정차구역,2020-03-17 14:02:57,서울 광진구 구의동 546-1
7,서울 광진구 구의동 546-11,37.535562,127.093359,광진구,지너스타워 주차장 앞,불법주정차구역,2020-03-17 14:02:57,서울 광진구 구의동 546-11
8,서울 광진구 구의동 611,37.538347,127.096620,광진구,현대2단지아파트 앞,불법주정차구역,2020-03-17 14:02:57,서울 광진구 구의동 611
9,서울 광진구 구의동 546-6,37.535668,127.094522,광진구,강변역 1번 출구,불법주정차구역,2020-03-17 14:02:57,서울 광진구 구의동 546-6


In [16]:
target_cctv_info['등록일시'] = pd.to_datetime(target_cctv_info['등록일시'])

In [17]:
target_cctv_info.to_csv('수정일자_이전_설치CCTV',index=False)

---

# Infra 관련 데이터프레임 종합

In [27]:
import os

# 폴더 경로 설정
folder_path = 'C:\\Users\\HP\\광진구\\인프라'

# 폴더 내의 모든 파일을 나열
file_names = os.listdir(folder_path)
infra_file_list = []
# 파일 이름 저장
for i in file_names:
    infra_file_list.append(i)

In [28]:
infra_file_list

['.ipynb_checkpoints',
 '서울시광진구골프연습장업인허가정보.csv',
 '서울시광진구공연장인허가정보.csv',
 '서울시광진구관광숙박업인허가정보.csv',
 '서울시광진구노래연습장업인허가정보.csv',
 '서울시광진구단란주점영업인허가정보.csv',
 '서울시광진구당구장업인허가정보.csv',
 '서울시광진구대규모점포인허가정보.csv',
 '서울시광진구동물병원인허가정보.csv',
 '서울시광진구목욕장업인허가정보.csv',
 '서울시광진구미용업인허가정보.csv',
 '서울시광진구민방위대피시설인허가정보.csv',
 '서울시광진구병원인허가정보.csv',
 '서울시광진구세탁업인허가정보.csv',
 '서울시광진구수영장업인허가정보.csv',
 '서울시광진구숙박업인허가정보.csv',
 '서울시광진구안전상비의약품판매업소인허가정보.csv',
 '서울시광진구약국인허가정보.csv',
 '서울시광진구영화상영관인허가정보.csv',
 '서울시광진구외국인관광도시민박업인허가정보.csv',
 '서울시광진구유흥주점영업인허가정보.csv',
 '서울시광진구의원인허가정보.csv',
 '서울시광진구일반음식점인허가정보.csv',
 '서울시광진구제과점영업인허가정보.csv',
 '서울시광진구집단급식소식품판매업인허가정보.csv',
 '서울시광진구집단급식소인허가정보.csv',
 '서울시광진구체육도장업인허가정보.csv',
 '서울시광진구휴게음식점인허가정보.csv',
 '서울특별시 광진구_가로등 위치정보_20220214.csv',
 '서울특별시 광진구_공영주차장 정보_20220118.csv',
 '서울특별시 광진구_주정차단속현황_20230331.csv']

In [29]:
def set_csv(file_name):
    import csv

    # CSV 파일을 읽기 모드로 열고 데이터를 리스트로 추출
    data = []
    if file_name[-3:] == 'csv':
        try:
            with open(folder_path+'\\'+file_name, 'r', encoding='cp949') as csv_file:
                csv_reader = csv.reader(csv_file)
                for row in csv_reader:
                    data.append(row)

            # 추출한 데이터를 Pandas DataFrame으로 변환
        except:
            with open(folder_path+'\\'+file_name, 'r', encoding='utf-8') as csv_file:
                csv_reader = csv.reader(csv_file)
                for row in csv_reader:
                    data.append(row)
            
        test = pd.DataFrame(data)
        test.columns  = test.iloc[0]
        test.drop(0,inplace=True)
        test['정보'] = file_name
        return test.to_csv(file_name,index=False)
    else :
        pass

In [21]:
#for i in infra_file_list:
#    set_csv(i)
# #데이터 프레임 정리

1. 인허가 일자 datetime 속성으로 변경
2. X좌표, Y좌표 찾아서 넣기
3. 정보 열 살리기

In [22]:
def modify_csv(test_df):
        #데이터 프레임 전처리 
        test_df['인허가일자'] = pd.to_datetime(test_df['인허가일자'])
        # datetime 속성 변환
        test_df['좌표정보(X)'] =  pd.to_numeric(test_df['좌표정보(X)'], errors='coerce')
        # 숫자 아니면 NaN 값 처리
        test_df['좌표정보(Y)'] = pd.to_numeric(test_df['좌표정보(Y)'], errors='coerce')
        # 숫자 아니면 NaN 값 처리
        idx = np.where(test_df.columns == '좌표정보(Y)')[0]
        # 좌표정보 Y의 인덱스 찾기
        for i, row in test_df.iterrows():
            if 200000 <= row['좌표정보(Y)'] < 300000:
                # 현재 '좌표정보Y' 값을 '좌표정보X'에 할당
                test_df.at[i, '좌표정보(X)'] = row['좌표정보(Y)']

                # '다음열'의 값을 '좌표정보Y'에 할당
                test_df.at[i, '좌표정보(Y)'] = row[test_df.columns[idx+1][0]]
                
        from pyproj import Proj, Transformer
        # 좌표 변환
        proj_korea = Proj('epsg:5174')  
        proj_wgs84 = Proj('epsg:4326')
        x_coord = test_df['좌표정보(X)']
        y_coord = test_df['좌표정보(Y)']

        transformer = Transformer.from_proj(proj_korea, proj_wgs84)
        longitude, latitude = transformer.transform(y_coord, x_coord)

        test_df['좌표정보(X)']=  latitude
        test_df['좌표정보(Y)']= longitude
        return test_df
def setting_csv_modify(file_path):
    # 인프라 폴더 내 파일 경로를 통해 csv 파일 읽어오기
    # 본인 폴더 및 파일 경로에 맞게 수정할 것
    try:
        test_df = pd.read_csv('인프라\\'+f'{file_path}',encoding='utf-8')
        modify_csv(test_df)
        test_df = test_df[['인허가일자','좌표정보(X)','좌표정보(Y)','폐업일자','영업상태명','정보']]
        test_df.to_csv(f'C:\\Users\\HP\\광진구\\수정_인프라\\{file_path}',index=False)
    except:
        test_df = pd.read_csv('인프라\\'+f'{file_path}',encoding='cp949')
        modify_csv(test_df)
        test_df = test_df[['인허가일자','좌표정보(X)','좌표정보(Y)','폐업일자','영업상태명','정보']]
        test_df.to_csv(f'C:\\Users\\HP\\광진구\\수정_인프라\\{file_path}',index=False)

In [23]:
for i in infra_file_list[1:-3]:
    setting_csv_modify(i)
# 밑에서 3개는 별도의 처리 필요함

C:\Users\HP\AppData\Local\Temp\ipykernel_2916\114912991.py:36: DtypeWarning: Columns (32,33,34,35,37,38,40,41,42,43,44,45,47,48,51) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv('인프라\\'+f'{file_path}',encoding='utf-8')


# 수정_인프라 폴더의 모든 DF를 하나의 메타 데이터로 합치기

In [24]:
import os
import pandas as pd

# 폴더 경로 설정
# 각자 맞는 폴더 경로 설정 필요
folder_path = 'C:\\Users\\HP\\광진구\\수정_인프라'

# 해당 폴더 내 모든 파일 리스트
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# 모든 데이터프레임을 저장할 리스트
df_list = []

# 파일을 하나씩 반복 처리
for file in files:
    # 파일 경로
    file_path = os.path.join(folder_path, file)
    # 파일 확장자 확인 (.csv인 경우만 처리)
    if file_path.endswith('.csv'):
        # 파일 읽기
        df = pd.read_csv(file_path)
        # 리스트에 추가
        df_list.append(df)

# 모든 데이터프레임을 하나로 병합
meta_df = pd.concat(df_list, ignore_index=True)

In [25]:
meta_df.to_csv('인프라_종합_데이터_3개제외',index=False)

In [26]:
meta_df

,인허가일자,좌표정보(X),좌표정보(Y),폐업일자,영업상태명,정보
0,1992-07-22,127.091354,37.538181,19990330,폐업,서울시광진구골프연습장업인허가정보.csv
1,1993-02-27,127.106577,37.547179,19980703,폐업,서울시광진구골프연습장업인허가정보.csv
2,1993-05-18,127.091249,37.537586,20230130,폐업,서울시광진구골프연습장업인허가정보.csv
3,1994-01-27,127.081989,37.555756,20080422,폐업,서울시광진구골프연습장업인허가정보.csv
4,1995-01-14,127.095874,37.538627,NaN,영업/정상,서울시광진구골프연습장업인허가정보.csv
...,...,...,...,...,...,...
30498,2023-10-25,127.061296,37.536253,NaN,영업/정상,서울시광진구휴게음식점인허가정보.csv
30499,2023-10-27,127.085362,37.557365,NaN,영업/정상,서울시광진구휴게음식점인허가정보.csv
30500,2023-11-02,127.078977,37.555599,NaN,영업/정상,서울시광진구휴게음식점인허가정보.csv
30501,2023-11-02,NaN,NaN,NaN,영업/정상,서울시광진구휴게음식점인허가정보.csv


## 메타 데이터 프레임을 통해 이후 격자 별 인프라 구성 요소를 매핑시킬 수 있음